# Install Dependencies

# Property Claim
We need to detect the damages of home or house, identify the reason for the damage

In [6]:
!pip install -Uqq fastbook

In [7]:
from fastbook import *

# Dataset

In [8]:
urls = search_images_ddg('bird photos', max_images=1)
len(urls), urls[0]

In [11]:
dest = Path('bird.jpg')
if not dest.exists(): download_url(urls[0], dest, show_progress=False)

In [10]:
doc(download_url)

In [12]:
im = Image.open(dest)
im.to_thumb(256, 256)

In [31]:
searches = 'forest', 'bird'
path = Path('bird_or_not')

In [30]:
# !mkdir bird_or_not

In [41]:
# for o in searches:
#     dest = (path/o)
#     dest.mkdir(exist_ok=True)
#     results = search_images_ddg(f'{o} photo')
#     download_images(dest, urls=results[:200])
#     resize_images(dest, max_size=400, dest=dest)

In [40]:
if not path.exists():
    parent = (path)
    parent.mkdir(exist_ok=True)
    for o in searches:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_ddg(f'{o} photo')
        download_images(dest, urls=results[:200])
        resize_images(dest, max_size=400, dest=dest)

# EDA

In [42]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)

# Training & Loss

In [43]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)

In [44]:
dls.show_batch(max_n=6)

In [45]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

# Predict

In [52]:
is_bird,_,probs = learn.predict(PILImage.create('bird.jpg'))
print(f"This is a: {is_bird}.")
print(f"Probability it's a bird: {probs[0]:.4f}")